In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)


2025-04-10 15:48:59.054383: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-10 15:48:59.055502: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-10 15:48:59.058585: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-10 15:48:59.067259: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744292939.082964  215913 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744292939.08

Source: https://www.reinfolib.mlit.go.jp/realEstatePrices/

In [2]:
data_dir = "~/Japanese_future_real_estate_prices/Data/2005_2024/trade_prices"

df_01 = pd.read_csv(f'{data_dir}/01_Hokkaido_20053_20243_en.csv',encoding="cp932", low_memory=False)

df_01.info()
df_01.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284244 entries, 0 to 284243
Data columns (total 29 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Type                              284244 non-null  object 
 1   Price information classification  284244 non-null  object 
 2   Area                              191138 non-null  object 
 3   City,Town,Ward,Village code       284244 non-null  int64  
 4   Prefecture                        284244 non-null  object 
 5   City,Town,Ward,Village            284244 non-null  object 
 6   District                          281231 non-null  object 
 7   Nearest station : Name            222269 non-null  object 
 8   Nearest station : Distance        218260 non-null  object 
 9   Total transaction value           284244 non-null  int64  
 10  Layout                            47031 non-null   object 
 11  Area(㎡)                           284244 non-null  o

,Type,Price information classification,Area,"City,Town,Ward,Village code",Prefecture,"City,Town,Ward,Village",District,Nearest station : Name,Nearest station : Distance,Total transaction value,...,Purpose of use,Frontage road : Direction,Frontage road : Type,Frontage road : Width,City planning,Building coverage ratio,Floor area ratio,Transaction timing,Renovation,Transaction factors
0,Residential Land(Land Only),Real Estate Transaction Price Information,Residential Area,1101,Hokkaido,"Chuo Ward, Sapporo City",Asahigaoka,Maruyama Koen,23,25000000,...,Office,East,Hokkaido Prefectural Road,15.0,1 Res,60.0,200.0,4th quarter 2023,NaN,NaN
1,Residential Land(Land Only),Real Estate Transaction Price Information,Residential Area,1101,Hokkaido,"Chuo Ward, Sapporo City",Asahigaoka,Maruyama Koen,23,16000000,...,House,West,City Road,6.0,1 Exc Low,40.0,80.0,4th quarter 2023,NaN,NaN
2,Residential Land(Land Only),Real Estate Transaction Price Information,Residential Area,1101,Hokkaido,"Chuo Ward, Sapporo City",Asahigaoka,Maruyama Koen,21,92000000,...,House,West,City Road,14.5,1 Res,60.0,200.0,2nd quarter 2023,NaN,NaN
3,Residential Land(Land and Building),Real Estate Transaction Price Information,Residential Area,1101,Hokkaido,"Chuo Ward, Sapporo City",Asahigaoka,Maruyama Koen,28,78000000,...,House,Northeast,Hokkaido Prefectural Road,15.0,1 Res,60.0,200.0,1st quarter 2023,NaN,NaN
4,Residential Land(Land and Building),Real Estate Transaction Price Information,Residential Area,1101,Hokkaido,"Chuo Ward, Sapporo City",Asahigaoka,Maruyama Koen,26,100000000,...,House,East,City Road,15.0,1 Res,60.0,200.0,1st quarter 2022,NaN,NaN


Let's start by removing columns that is not necessary for our neural network

In [3]:
# Remove unwanted columns #
df_01 = pd.read_csv(f'{data_dir}/01.csv',encoding="cp932", low_memory=False)
df_01.drop(columns=['Price information classification', 'City,Town,Ward,Village code', 'District', 'Nearest station : Name', 
                     'City planning', 'Land : Shape', 'Frontage road : Direction', 'Frontage road : Type', 'Frontage road : Width'], inplace=True)

#df_01['Renovation'].isna().sum() / len(df_01) more than 90% of values in the Renovation column is NaN
#df_01['Transaction factors'].isna().sum() / len(df_01) more than 90% of values in the Transaction factors column are NaN
#df_01['Layout'].isna().sum() / len(df_01) more than 83% of values in the Layout column are NaN
#df_01['Building : Structure'].isna().sum() / len(df_01) more than 52% of values in the Building : Structure column are NaN
df_01.drop(columns=['Renovation', 'Transaction factors', 'Layout', 'Building : Structure', 'Land : Price per ㎡'], inplace=True)


df_01.info()
df_01.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284244 entries, 0 to 284243
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Type                          284244 non-null  object 
 1   Area                          191138 non-null  object 
 2   Prefecture                    284244 non-null  object 
 3   City,Town,Ward,Village        284244 non-null  object 
 4   Nearest station : Distance    218260 non-null  object 
 5   Total transaction value       284244 non-null  int64  
 6   Area(㎡)                       284244 non-null  object 
 7   Frontage                      181423 non-null  object 
 8   Building : Total floor area   101238 non-null  object 
 9   Building : Construction year  137074 non-null  object 
 10  Use                           115246 non-null  object 
 11  Purpose of use                115860 non-null  object 
 12  Building coverage ratio       210995 non-nul

,Type,Area,Prefecture,"City,Town,Ward,Village",Nearest station : Distance,Total transaction value,Area(㎡),Frontage,Building : Total floor area,Building : Construction year,Use,Purpose of use,Building coverage ratio,Floor area ratio,Transaction timing
0,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",23,25000000,130,8,NaN,NaN,NaN,Office,60.0,200.0,4th quarter 2023
1,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",23,16000000,115,10,NaN,NaN,NaN,House,40.0,80.0,4th quarter 2023
2,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",21,92000000,410,14,NaN,NaN,NaN,House,60.0,200.0,2nd quarter 2023
3,Residential Land(Land and Building),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",28,78000000,340,13,300,1990,House,House,60.0,200.0,1st quarter 2023
4,Residential Land(Land and Building),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",26,100000000,340,13.4,300,1990,House,House,60.0,200.0,1st quarter 2022


In [4]:
# We only want to investigate real estate that is intended to be used as a house or has been a house

#Residential_condition = (df_01["Type"] != "Forest Land") & (df_01["Type"] != "Agricultural Land")

Intended_House_condition = (df_01['Purpose of use'] == "House") | (df_01['Use'] == "House")

House_df = df_01[Intended_House_condition]
House_df.drop(columns=['Purpose of use', 'Use'], inplace=True) # No longer needed
print(f'{len(House_df)} entries, ({(len(House_df) / len(df_01)) * 100} % of total) satisfy the housing condition.')

133464 entries, (46.954025414784475 % of total) satisfy the housing condition.


/tmp/ipykernel_215913/1645358163.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df.drop(columns=['Purpose of use', 'Use'], inplace=True) # No longer needed


In [5]:
# Any type that is Land Only will not have a floor size, so we can set the TotalFloorArea to -1. Same logic for other Building stats and frontage
House_df.loc[House_df['Building : Total floor area'].isna() & (House_df['Type'] == 'Residential Land(Land Only)'), 'Building : Total floor area'] = -1
House_df.loc[House_df['Building : Construction year'].isna() & (House_df['Type'] == 'Residential Land(Land Only)'), 'Building : Construction year'] = -1
House_df.loc[House_df['Building coverage ratio'].isna() & (House_df['Type'] == 'Residential Land(Land Only)'), 'Building coverage ratio'] = -1
House_df.loc[House_df['Floor area ratio'].isna() & (House_df['Type'] == 'Residential Land(Land Only)'), 'Floor area ratio'] = -1
House_df.loc[House_df['Frontage'].isna() & (House_df['Type'] == 'Residential Land(Land Only)'), 'Frontage'] = -1
House_df.loc[House_df['Frontage'].isna() & (House_df['Type'] == 'Residential Land(Land Only)'), 'Frontage'] = -1

#If the region or time to station cannot be determined, delete
# Drop rows where 'Region' or time to station has NaN values
House_df.dropna(subset=['Area', 'Nearest station : Distance'], inplace=True)
House_df.info()
House_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 99152 entries, 1 to 281618
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Type                          99152 non-null  object 
 1   Area                          99152 non-null  object 
 2   Prefecture                    99152 non-null  object 
 3   City,Town,Ward,Village        99152 non-null  object 
 4   Nearest station : Distance    99152 non-null  object 
 5   Total transaction value       99152 non-null  int64  
 6   Area(㎡)                       99152 non-null  object 
 7   Frontage                      96683 non-null  object 
 8   Building : Total floor area   96816 non-null  object 
 9   Building : Construction year  95744 non-null  object 
 10  Building coverage ratio       96203 non-null  float64
 11  Floor area ratio              96203 non-null  float64
 12  Transaction timing            99152 non-null  object 
dtypes: fl

/tmp/ipykernel_215913/1812029588.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df.dropna(subset=['Area', 'Nearest station : Distance'], inplace=True)


,Type,Area,Prefecture,"City,Town,Ward,Village",Nearest station : Distance,Total transaction value,Area(㎡),Frontage,Building : Total floor area,Building : Construction year,Building coverage ratio,Floor area ratio,Transaction timing
1,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",23,16000000,115,10,-1,-1,40.0,80.0,4th quarter 2023
2,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",21,92000000,410,14,-1,-1,60.0,200.0,2nd quarter 2023
3,Residential Land(Land and Building),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",28,78000000,340,13,300,1990,60.0,200.0,1st quarter 2023
4,Residential Land(Land and Building),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",26,100000000,340,13.4,300,1990,60.0,200.0,1st quarter 2022
5,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",26,32000000,210,19.2,-1,-1,40.0,80.0,4th quarter 2021


In [6]:
House_df['Quarter'] = House_df['Transaction timing'].str.extract(r'(\d)')[0].astype(int)
House_df['Year'] = House_df['Transaction timing'].str.extract(r'(\d{4})')[0].astype(int)
House_df.drop(columns=['Transaction timing'], inplace=True)
House_df.info()
House_df.head()

/tmp/ipykernel_215913/3097317902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Quarter'] = House_df['Transaction timing'].str.extract(r'(\d)')[0].astype(int)


<class 'pandas.core.frame.DataFrame'>
Index: 99152 entries, 1 to 281618
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Type                          99152 non-null  object 
 1   Area                          99152 non-null  object 
 2   Prefecture                    99152 non-null  object 
 3   City,Town,Ward,Village        99152 non-null  object 
 4   Nearest station : Distance    99152 non-null  object 
 5   Total transaction value       99152 non-null  int64  
 6   Area(㎡)                       99152 non-null  object 
 7   Frontage                      96683 non-null  object 
 8   Building : Total floor area   96816 non-null  object 
 9   Building : Construction year  95744 non-null  object 
 10  Building coverage ratio       96203 non-null  float64
 11  Floor area ratio              96203 non-null  float64
 12  Quarter                       99152 non-null  int64  
 13  Year 

/tmp/ipykernel_215913/3097317902.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Year'] = House_df['Transaction timing'].str.extract(r'(\d{4})')[0].astype(int)
/tmp/ipykernel_215913/3097317902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df.drop(columns=['Transaction timing'], inplace=True)


,Type,Area,Prefecture,"City,Town,Ward,Village",Nearest station : Distance,Total transaction value,Area(㎡),Frontage,Building : Total floor area,Building : Construction year,Building coverage ratio,Floor area ratio,Quarter,Year
1,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",23,16000000,115,10,-1,-1,40.0,80.0,4,2023
2,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",21,92000000,410,14,-1,-1,60.0,200.0,2,2023
3,Residential Land(Land and Building),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",28,78000000,340,13,300,1990,60.0,200.0,1,2023
4,Residential Land(Land and Building),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",26,100000000,340,13.4,300,1990,60.0,200.0,1,2022
5,Residential Land(Land Only),Residential Area,Hokkaido,"Chuo Ward, Sapporo City",26,32000000,210,19.2,-1,-1,40.0,80.0,4,2021


In [7]:
House_df['Prefecture'].replace('Hokkaido', 0, inplace=True)
capital_dict = {0:"Sapporo"}
def categorize_municipality(municipality, prefecture):
    if 'Sapporo' in municipality:
        return 4  # Capital
    elif 'City' in municipality:
        return 3  # City
    elif 'Town' in municipality:
        return 2  # Town
    elif 'Village' in municipality:
        return 1  # Village
    else:
        return 0  # Default

# Apply the categorization function
House_df['MunicipalityCategory'] = House_df.apply(lambda row: categorize_municipality(row['City,Town,Ward,Village'], row['Prefecture']), axis=1)
House_df.head()

/tmp/ipykernel_215913/1020006080.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  House_df['Prefecture'].replace('Hokkaido', 0, inplace=True)
/tmp/ipykernel_215913/1020006080.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  House_df['Prefecture'].replace('Hokkaido', 0, inplace=True)
/tmp/ipyke

,Type,Area,Prefecture,"City,Town,Ward,Village",Nearest station : Distance,Total transaction value,Area(㎡),Frontage,Building : Total floor area,Building : Construction year,Building coverage ratio,Floor area ratio,Quarter,Year,MunicipalityCategory
1,Residential Land(Land Only),Residential Area,0,"Chuo Ward, Sapporo City",23,16000000,115,10,-1,-1,40.0,80.0,4,2023,4
2,Residential Land(Land Only),Residential Area,0,"Chuo Ward, Sapporo City",21,92000000,410,14,-1,-1,60.0,200.0,2,2023,4
3,Residential Land(Land and Building),Residential Area,0,"Chuo Ward, Sapporo City",28,78000000,340,13,300,1990,60.0,200.0,1,2023,4
4,Residential Land(Land and Building),Residential Area,0,"Chuo Ward, Sapporo City",26,100000000,340,13.4,300,1990,60.0,200.0,1,2022,4
5,Residential Land(Land Only),Residential Area,0,"Chuo Ward, Sapporo City",26,32000000,210,19.2,-1,-1,40.0,80.0,4,2021,4


In [8]:
House_df[House_df.isna().any(axis=1)]
House_df.dropna(inplace=True)
House_df

/tmp/ipykernel_215913/4175586925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df.dropna(inplace=True)


,Type,Area,Prefecture,"City,Town,Ward,Village",Nearest station : Distance,Total transaction value,Area(㎡),Frontage,Building : Total floor area,Building : Construction year,Building coverage ratio,Floor area ratio,Quarter,Year,MunicipalityCategory
1,Residential Land(Land Only),Residential Area,0,"Chuo Ward, Sapporo City",23,16000000,115,10,-1,-1,40.0,80.0,4,2023,4
2,Residential Land(Land Only),Residential Area,0,"Chuo Ward, Sapporo City",21,92000000,410,14,-1,-1,60.0,200.0,2,2023,4
3,Residential Land(Land and Building),Residential Area,0,"Chuo Ward, Sapporo City",28,78000000,340,13,300,1990,60.0,200.0,1,2023,4
4,Residential Land(Land and Building),Residential Area,0,"Chuo Ward, Sapporo City",26,100000000,340,13.4,300,1990,60.0,200.0,1,2022,4
5,Residential Land(Land Only),Residential Area,0,"Chuo Ward, Sapporo City",26,32000000,210,19.2,-1,-1,40.0,80.0,4,2021,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281599,Residential Land(Land and Building),Residential Area,0,"Shiranuka Town, Shiranuka County",2,7500000,440,29,85,2000,60.0,200.0,1,2016,2
281601,Residential Land(Land Only),Residential Area,0,"Shiranuka Town, Shiranuka County",7,1100000,440,17,-1,-1,60.0,200.0,4,2016,2
281603,Residential Land(Land and Building),Residential Area,0,"Shiranuka Town, Shiranuka County",17,1000000,300,20,100,1980,60.0,200.0,2,2016,2
281606,Residential Land(Land and Building),Commercial Area,0,"Shiranuka Town, Shiranuka County",4,1600000,250,9.1,230,1989,80.0,400.0,3,2016,2


In [9]:
# Replace '2,000 ㎡ or greater.' with 2000 in the 'Area(㎡)' column
House_df['Area(㎡)'] = House_df['Area(㎡)'].replace('2,000 ㎡ or greater.', 2000)

# Convert the 'Area(㎡)' column to numeric (if it's not already)
House_df['Area(㎡)'] = pd.to_numeric(House_df['Area(㎡)'], errors='coerce')

# Create a new column 'Area_greater_2000_flag' as True if area >= 2000, otherwise False
House_df['Area_greater_2000_flag'] = House_df['Area(㎡)'] >= 2000

/tmp/ipykernel_215913/1903490588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Area(㎡)'] = House_df['Area(㎡)'].replace('2,000 ㎡ or greater.', 2000)
/tmp/ipykernel_215913/1903490588.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Area(㎡)'] = pd.to_numeric(House_df['Area(㎡)'], errors='coerce')
/tmp/ipykernel_215913/1903490588.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [10]:
# Replace 'before the war' with 1945 in the 'Building : Construction year' column
House_df['Building : Construction year'] = House_df['Building : Construction year'].replace('before the war', 1945)

# Convert the 'Building : Construction year' column to numeric (if it's not already)
House_df['Building : Construction year'] = pd.to_numeric(House_df['Building : Construction year'], errors='coerce')

# Create a new column 'before_the_war_flag' as True if the year <= 1945, otherwise False
House_df['before_the_war_flag'] = House_df['Building : Construction year'] <= 1945


/tmp/ipykernel_215913/904521495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Building : Construction year'] = House_df['Building : Construction year'].replace('before the war', 1945)
/tmp/ipykernel_215913/904521495.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Building : Construction year'] = pd.to_numeric(House_df['Building : Construction year'], errors='coerce')
/tmp/ipykernel_215913/904521495.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [18]:
import re


def convert_to_minutes(distance_str):
    # If it contains a range (e.g., '30-60minutes' or '1H-1H30')
    if '-' in distance_str:
        print(distance_str)
        if 'H' in distance_str:
            # Split the range by "-"
            parts = distance_str.split('-')
        
            # Split each part by "H" and handle hours and minutes
            start = parts[0].split('H')
            end = parts[1].split('H')
            print(start, end)
            # Convert start time
            start_hours = int(start[0]) if start[0] else 0
            start_minutes = int(start[1]) if len(start) > 1 and start[1]  else 0
            start_total_minutes = start_hours * 60 + start_minutes
            
            # Convert end time
            end_hours = int(end[0]) if end[0] else 0
            end_minutes = int(end[1]) if len(end) > 1 and end[1] else 0
            end_total_minutes = end_hours * 60 + end_minutes
            
            if end_hours == 0 and end_minutes == 0: #deal with edge cases such as "2H-"
                return start_total_minutes
            # Return the average of the two
            return (start_total_minutes + end_total_minutes) // 2
        
        elif 'minutes' in distance_str:
            parts = distance_str.split('-')

            start = int(parts[0])
            end = int(parts[1].split("minutes")[0])

            return (start + end) // 2
    
    else:  # If there's only a single value (e.g., '30minutes')
        # Extract the number and convert to minutes (if needed)
        if 'H' in distance_str:  # In case it's in hours
            hours = int(re.search(r'(\d+)', distance_str).group(0))
            return hours * 60  # Convert to minutes
        else:  # Just minutes
            return int(re.search(r'(\d+)', distance_str).group(0))


# Apply the function to the column
House_df['Average Distance to Station'] = House_df['Nearest station : Distance'].apply(convert_to_minutes)
House_df.drop(columns=['Nearest station : Distance'], inplace=True)
print(max(House_df['Average Distance to Station']))

30-60minutes
1H30-2H
['1', '30'] ['2', '']
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60minutes
30-60min

In [13]:
# Replace "less than 10 ㎡" with 10 and "2,000 ㎡ or greater" with 2000
House_df['Building : Total floor area'] = House_df['Building : Total floor area'].replace({
    'less than 10 ㎡': 10,
    '2,000 ㎡ or greater': 2000
})

# Convert the 'Building : Total floor area' column to numeric (if it's not already)
House_df['Building : Total floor area'] = pd.to_numeric(House_df['Building : Total floor area'], errors='coerce')

# Create a new column 'floor_area_greater_than_2000' as True if area >= 2000, otherwise False
House_df['floor_area_greater_than_2000'] = House_df['Building : Total floor area'] >= 2000


/tmp/ipykernel_215913/2482399433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Building : Total floor area'] = House_df['Building : Total floor area'].replace({
/tmp/ipykernel_215913/2482399433.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Building : Total floor area'] = pd.to_numeric(House_df['Building : Total floor area'], errors='coerce')
/tmp/ipykernel_215913/2482399433.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [14]:
# Replace '50.0m or longer.' with 50 in the 'Frontage' column
House_df['Frontage'] = House_df['Frontage'].replace('50.0m or longer.', 50)

# Convert the 'Frontage' column to numeric (if it's not already)
House_df['Frontage'] = pd.to_numeric(House_df['Frontage'], errors='coerce')

/tmp/ipykernel_215913/1509787632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Frontage'] = House_df['Frontage'].replace('50.0m or longer.', 50)
/tmp/ipykernel_215913/1509787632.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  House_df['Frontage'] = pd.to_numeric(House_df['Frontage'], errors='coerce')


In [24]:
House_df['Area'].unique()

region_encoded = pd.get_dummies(House_df['Area'], prefix='Region')
# Merge with original dataframe
House_df = pd.concat([House_df, region_encoded], axis=1)

# Drop the original 'Type' column
House_df.drop(columns=['Area'], inplace=True)

House_df

,Type,Prefecture,Locating,TotalTransactionValue,Frontage,TotalFloorArea,ConstructionYear,BuildingCoverageRatio,FloorAreaRatio,Quarter,...,Region_1100,Region_1200,Region_1300,Region_1400,Region_1500,Region_1600,Region_1700,Region_1800,Region_1900,Region_2000
1,Residential Land(Land Only),0,"Chuo Ward, Sapporo City",16000000,10.0,-1.0,-1,40.0,80.0,4,...,False,False,False,False,False,False,False,False,False,False
2,Residential Land(Land Only),0,"Chuo Ward, Sapporo City",92000000,14.0,-1.0,-1,60.0,200.0,2,...,False,False,False,False,False,False,False,False,False,False
3,Residential Land(Land and Building),0,"Chuo Ward, Sapporo City",78000000,13.0,300.0,1990,60.0,200.0,1,...,False,False,False,False,False,False,False,False,False,False
4,Residential Land(Land and Building),0,"Chuo Ward, Sapporo City",100000000,13.4,300.0,1990,60.0,200.0,1,...,False,False,False,False,False,False,False,False,False,False
5,Residential Land(Land Only),0,"Chuo Ward, Sapporo City",32000000,19.2,-1.0,-1,40.0,80.0,4,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281599,Residential Land(Land and Building),0,"Shiranuka Town, Shiranuka County",7500000,29.0,85.0,2000,60.0,200.0,1,...,False,False,False,False,False,False,False,False,False,False
281601,Residential Land(Land Only),0,"Shiranuka Town, Shiranuka County",1100000,17.0,-1.0,-1,60.0,200.0,4,...,False,False,False,False,False,False,False,False,False,False
281603,Residential Land(Land and Building),0,"Shiranuka Town, Shiranuka County",1000000,20.0,100.0,1980,60.0,200.0,2,...,False,False,False,False,False,False,False,False,False,False
281606,Residential Land(Land and Building),0,"Shiranuka Town, Shiranuka County",1600000,9.1,230.0,1989,80.0,400.0,3,...,False,False,False,False,False,False,False,False,False,False


In [16]:
# Define the dictionary mapping old column names to new ones
column_mapping = {
    'City,Town,Ward,Village': 'Location',
    'Total transaction value': 'TotalTransactionValue',
    'Area(㎡)': 'Area',
    'Building : Total floor area': 'TotalFloorArea',
    'Building : Construction year': 'ConstructionYear',
    'Building coverage ratio': 'BuildingCoverageRatio',
    'Floor area ratio': 'FloorAreaRatio',
    'Area_greater_2000_flag': 'AreaGreaterFlag',
    'before_the_war_flag': 'BeforeWarFlag',
    'Average Distance to Station': 'AverageTimeToStation',
    'floor_area_greater_than_2000': 'FloorAreaGreaterFLag',
    'frontage_greater_than_50' : 'FrontageGreaterFlag',
    'Region_Commercial Area': 'RegionCommercialArea',
    'Region_Industrial Area': 'RegionIndustrialArea',
    'Region_Potential Residential Area': 'RegionPotentialResidentialArea',
    'Region_Residential Area': 'RegionResidentialArea'
}

# Rename columns using the dictionary
House_df = House_df.rename(columns=column_mapping)

# Display the updated DataFrame
print(House_df.columns)


Index(['Type', 'Prefecture', 'Locating', 'Nearest station : Distance',
       'TotalTransactionValue', 'Area', 'Frontage', 'TotalFloorArea',
       'ConstructionYear', 'BuildingCoverageRatio', 'FloorAreaRatio',
       'Quarter', 'Year', 'MunicipalityCategory', 'AreaGreaterFlag',
       'BeforeWarFlag', 'AverageTimeToStation', 'FloorAreaGreaterFLag',
       'RegionCommercialArea', 'RegionIndustrialArea',
       'RegionPotentialResidentialArea', 'RegionResidentialArea'],
      dtype='object')


In [23]:
House_df['Area'].unique()

array([ 115,  410,  340,  210,  175,  140,  290,   60,  260,  165,  185,
        240,  500,  310,  155,  390,  145,  430,  370,  350,  400,  730,
        270,  530,  520,  195,  200,   90,  130,  610,  300,  220,  620,
        330,  250,  810,  230,   65,  180,  100,  110,  280,   80,  170,
        120,  135,   70,  580,  760, 1100, 1600,  800,  150,  460,   95,
         85,  650,  450,  125,  600,  660,   40,  630,  360,  160, 2000,
        380, 1200,   75,  480,  680,  420,  840,  320,  880,  540,  560,
        190, 1400,  440,  105, 1700, 1000,  640,  940,  590,  690,  790,
        990,  740,  490,  970, 1500,  780,  550,  510,  750, 1300,  470,
         50,  700,  950,  570,  770,  670,  980,  720,   55, 1800,  850,
        710,   45,  900,  860,  870,  960,   35,  830,  920,  820,  930,
        910, 1900,  890,   30])

In [113]:
# Save the DataFrame to a CSV file
House_df.to_csv('/home/brianmcgloughlin/topcat_hokkaido.csv', index=False)
